Using SX generate parquet files for testing with join infrastructure. Also, using new SX frontend.

Constants:

In [1]:
# A Z to ee sample - Release 21
ds_name = r"mc16_13TeV:mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.deriv.DAOD_PHYS.e3601_e5984_s3126_r10201_r10210_p5313"

What we'll need for imports

In [2]:
from servicex_client.servicex_client import ServiceXClient
from servicex_client.dataset_identifier import RucioDatasetIdentifier
from servicex_client.models import ResultFormat
from func_adl_servicex_xaodr22.event_collection import Event

# import logging
# logging.basicConfig(level=logging.DEBUG)

In [5]:
sx = ServiceXClient(backend="test4")
did = RucioDatasetIdentifier(ds_name, num_files=10)
ds_raw = sx.func_adl_dataset(
    did, codegen="atlasr21", title="Zee", result_format=ResultFormat.parquet, item_type=Event
)

from func_adl_servicex_xaodr22 import calib_tools
#ds = calib_tools.apply_calibration(ds_raw, "PHYS")
ds = calib_tools.query_update(ds_raw, calib_tools.default_config("PHYSLITE"))

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='c:\\Users\\gordo\\Code\\iris-hep\\joiner\\.venv\\lib\\site-packages\\certifi\\cacert.pem'


DEBUG:httpcore.connection:connect_tcp.started host='servicex-release-testing-4.servicex.ssl-hep.org' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x00000264CE030940>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x00000264CE11CFC0> server_hostname='servicex-release-testing-4.servicex.ssl-hep.org' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x00000264CE030460>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'H

In [6]:
electrons = (
    ds.Select(lambda e: e.Electrons('Electrons'))
    .Select(
        lambda e: e.Where(lambda e: (e.pt() / 1000 > 25.0) and (abs(e.eta()) < 2.5))
    )
    .Select(lambda e: e.Select(lambda e: e.pt() / 1000))
).as_files() #.generate_selection_string()
electrons

INFO:func_adl.object_stream:Overwriting metadata "calibration" from its old value of "CalibrationEventConfig(jet_collection='AnalysisJets', jet_calib_truth_collection='AntiKt4TruthDressedWZJets', run_jet_ghost_muon_association=False, electron_collection='AnalysisElectrons', electron_working_point='MediumLHElectron', electron_isolation='NonIso', photon_collection='AnalysisPhotons', photon_working_point='Tight', photon_isolation='FixedCutTight', muon_collection='AnalysisMuons', muon_working_point='Medium', muon_isolation='NonIso', tau_collection='AnalysisTaus', tau_working_point='Tight', perform_overlap_removal=True, datatype='mc', correct_pileup=True, calibrate=False, uncalibrated_possible=False)" to "CalibrationEventConfig(jet_collection='AnalysisJets', jet_calib_truth_collection='AntiKt4TruthDressedWZJets', run_jet_ghost_muon_association=False, electron_collection='Electrons', electron_working_point='MediumLHElectron', electron_isolation='NonIso', photon_collection='AnalysisPhotons', 

Output()

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='c:\\Users\\gordo\\Code\\iris-hep\\joiner\\.venv\\lib\\site-packages\\certifi\\cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='servicex-release-testing-4.servicex.ssl-hep.org' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x00000264CE12ADA0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x00000264CE099440> server_hostname='servicex-release-testing-4.servicex.ssl-hep.org' timeout=5.0


DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x00000264CE0F89A0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 12 Jul 2023 23:56:34 GMT'), (b'Content-Type', b'application/json'), (b'Content-Length', b'332'), (b'Connection', b'keep-alive'), (b'Access-Control-Allow-Origin', b'*'), (b'Strict-Transport-Security', b'max-age=15724800; includeSubDomains')])
INFO:httpx:HTTP Request: POST https://servicex-release-testing-4.servicex.ssl-hep.org//token/refresh "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 12 Jul 2023 23:56:34 GMT'), (b'Content-Type', b'application/json'), (b'Content-Length', b'55'), (b'Connection', b'keep-alive'), (b'Access-Control-Allow-Origin', b'*'), (b'Strict-Transport-Security', b'max-age=15724800; includeSubDomains')])
INFO:httpx:HTTP Request: POST https://servicex-release-testing-4.servicex.ssl-hep.org//servicex/transformation "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='c:\\Users\\gordo\\Code\\iris-hep\\joiner\\.venv\\lib\\site-packages\\certifi\\cacert.pem'


...

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 12 Jul 2023 23:57:36 GMT'), (b'Content-Type', b'application/json'), (b'Content-Length', b'332'), (b'Connection', b'keep-alive'), (b'Access-Control-Allow-Origin', b'*'), (b'Strict-Transport-Security', b'max-age=15724800; includeSubDomains')])
INFO:httpx:HTTP Request: POST https://servicex-release-testing-4.servicex.ssl-hep.org//token/refresh "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers

Transforms completed successfully

DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.


TransformedResults(hash='311aecb8f5ce1a2046732df94f42b27909b8c112a53fbddb7e3fb9e7435eaecb', title='Zee', codegen='atlasr21', request_id='ba35edd9-96b0-48dd-af6d-2f4cb4c18502', submit_time=datetime.datetime(2023, 7, 12, 23, 56, 34, 145647, tzinfo=datetime.timezone.utc), data_dir='sx_cache/ba35edd9-96b0-48dd-af6d-2f4cb4c18502', file_list=['C:/Users/gordo/Code/iris-hep/joiner/sx_cache/ba35edd9-96b0-48dd-af6d-2f4cb4c18502/_1fef7c829002f17c7cf6b6040ae8b1bc8d53ef6c8.pool.root.parquet', 'C:/Users/gordo/Code/iris-hep/joiner/sx_cache/ba35edd9-96b0-48dd-af6d-2f4cb4c18502/_5ea15726988e2cabe77ef8ff3060e301abb4adad3.pool.root.parquet', 'C:/Users/gordo/Code/iris-hep/joiner/sx_cache/ba35edd9-96b0-48dd-af6d-2f4cb4c18502/_fea60efda9349ddcf67332e74684c32fb0e1279e1.pool.root.parquet', 'C:/Users/gordo/Code/iris-hep/joiner/sx_cache/ba35edd9-96b0-48dd-af6d-2f4cb4c18502/_300acfa59326ff8ec94539acc44013a581d700593.pool.root.parquet', 'C:/Users/gordo/Code/iris-hep/joiner/sx_cache/ba35edd9-96b0-48dd-af6d-2f4cb4c

DEBUG:Comm:handle_msg[c0cc28c97e40437e84dfb4793fa95973]({'header': {'date': datetime.datetime(2023, 7, 12, 23, 57, 36, 777000, tzinfo=tzutc()), 'msg_id': '6c2a98be-e333-4281-95f5-d0675f227a80', 'msg_type': 'comm_msg', 'session': '1eaa59f2-84d8-4a20-8b0f-9f93df52cb63', 'username': '1872b749-2a1f-4992-9fe9-cc7f56e4daa9', 'version': '5.2'}, 'msg_id': '6c2a98be-e333-4281-95f5-d0675f227a80', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': 'c0cc28c97e40437e84dfb4793fa95973', 'data': {'method': 'update', 'state': {'outputs': [{'output_type': 'display_data', 'data': {'text/plain': 'Transform \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b10/10\x1b \x1b01:01\x1b\nDownload  \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b10/10\x1b \x1b00:57\x1b\n', 'text/html': '<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,\'DejaVu Sans Mono\',consolas,\'Courier New\',monospace">Transform <span style="color: #729c1f; text-decorati

In [ ]:
import awkward as ak
pts = ak.from_parquet(electrons.signed_url_list)

# Query 1 - Electrons

Good healthy electrons that should be part of a Z mass peak: